In [8]:
# Luis Gabriel Delfín Paulín
# A01701482

# Paso 1: Leer matrices de CSV (un archivo por figura)
def read_csv_matrix(path):
    mat = []
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            row = [int(x) for x in line.split(",")]
            mat.append(row)
    return mat

patterns = {
    "arrow_up":       read_csv_matrix("arrow_up.csv"),
    "triangle_yield": read_csv_matrix("triangle_yield.csv"),
    "no_entry":       read_csv_matrix("no_entry.csv"),
    "parking_sign":   read_csv_matrix("parking_sign.csv"),
    "bicycle_sign":   read_csv_matrix("bicycle_sign.csv"),
}

arrow_up_noisy = read_csv_matrix("arrow_up_noisy.csv")

sizes = {(len(m), len(m[0])) for m in patterns.values()}
assert len(sizes) == 1, "Todas las figuras deben tener el MISMO tamaño."
M, N = next(iter(sizes))
assert M <= 10 and N <= 10, "El tamaño máximo permitido es 10x10."
assert len(arrow_up_noisy) == M and len(arrow_up_noisy[0]) == N, "arrow_up_noisy.csv debe tener el mismo tamaño que los demás."

print("# Patrones (0/1) cargados de CSV:")
for name, mat in patterns.items():
    print(name)
    for r in mat: print("".join("#" if v else "." for v in r))
    print()

# Paso 2: Convertir a vectores ±1
def flatten(mat):
    return [v for row in mat for v in row]

def to_pm1(mat01):
    return [1 if v==1 else -1 for v in flatten(mat01)]

vecs = {name: to_pm1(mat) for name, mat in patterns.items()}
names = list(vecs.keys())
x0 = vecs[names[0]]
n  = len(x0)

# Paso 3: Calcular outer products y sumar (Hebb)
def outer(v):
    nn = len(v)
    Mtx = [[0]*nn for _ in range(nn)]
    for i in range(nn):
        vi = v[i]
        for j in range(nn):
            Mtx[i][j] = vi * v[j]
    return Mtx

def mat_sum(A, B):
    nn = len(A); mm = len(A[0])
    C = [[0]*mm for _ in range(nn)]
    for i in range(nn):
        Ai=A[i]; Bi=B[i]; Ci=C[i]
        for j in range(mm):
            Ci[j] = Ai[j] + Bi[j]
    return C

first = True
for nm in names:
    if first:
        T = [row[:] for row in outer(vecs[nm])]
        first = False
    else:
        T = mat_sum(T, outer(vecs[nm]))

# Paso 4: poner diagonal en 0
for i in range(n):
    T[i][i] = 0

# Paso 5: Hopfield
def log2_approx(x):
    if x <= 0: return 0.0
    k = 0
    p = 1.0
    while p*2.0 <= x:
        p *= 2.0
        k += 1
    frac = (x - p) / p
    return k + frac / 2.0

p1 = (15 * n) / 100.0
lg = log2_approx(float(n))
p2 = n / (2.0 * lg) if lg > 0 else 0
p_safe = int(p2 if p2 < p1 else p1)

# Paso 6: Reconocer patrón
def matvec_row(row_vec, Mtx):
    nn = len(row_vec)
    out = [0]*nn
    for j in range(nn):
        s = 0
        for k in range(nn):
            s += row_vec[k] * Mtx[k][j]
        out[j] = s
    return out

def step_keep(x, prev):
    if x > 0: return 1
    if x < 0: return -1
    return prev

def run_until_fix(state_pm1, T, max_iters=25):
    s = state_pm1[:]
    for _ in range(max_iters):
        z = matvec_row(s, T)
        new_s = [step_keep(z[i], s[i]) for i in range(len(s))]
        if new_s == s:
            break
        s = new_s
    return s

def unflatten_pm1_to01(vec_pm1, M, N):
    z = [1 if x==1 else 0 for x in vec_pm1]
    return [z[i*N:(i+1)*N] for i in range(M)]

def hamming(a,b):
    d=0
    for x,y in zip(a,b):
        if x!=y: d+=1
    return d

def recognize_matrix(mat01):
    v0 = to_pm1(mat01)
    vf = run_until_fix(v0, T)
    best_name=None; best_d=10**9
    for nm, ref in vecs.items():
        d = hamming(vf, ref)
        if d < best_d:
            best_d, best_name = d, nm
    return best_name, unflatten_pm1_to01(vf, M, N)

# Paso 7: Probar reconocimiento (usando lo cargado de CSV)
for nm, mat in patterns.items():
    pred, rec = recognize_matrix(mat)
    print("Entrada:", nm, "-> Predicción:", pred)
    for r in rec: print("".join("#" if v else "." for v in r))
    print()

# Paso 8: Probar el archivo con ruido (no entrenado)
pred_noisy, rec_noisy = recognize_matrix(arrow_up_noisy)
print("Entrada: arrow_up_noisy -> Predicción:", pred_noisy)
for r in rec_noisy: print("".join("#" if v else "." for v in r))
print()


# Patrones (0/1) cargados de CSV:
arrow_up
....#.....
...###....
..#.#.#...
.#..#..#..
#...#...#.
....#.....
....#.....
....#.....
....#.....
....#.....

triangle_yield
.....#....
....###...
...#####..
..#######.
.#########
..........
..........
..........
..........
..........

no_entry
..######..
.#......#.
#........#
##########
##########
#........#
#........#
#........#
.#......#.
..######..

parking_sign
##########
##.......#
######...#
##...#...#
##...#...#
######...#
##.......#
##.......#
##########
##########

bicycle_sign
..######..
.#......#.
#.#....#.#
#..#..#..#
#...##...#
#..#..#..#
#.#....#.#
.#......#.
..######..
..........

# N = 100 neuronas (tamaño 10x10)
# Capacidad teórica: 0.15N≈15.0 | N/(2log2N)≈8.0 | Sugerida≈7

Entrada: arrow_up -> Predicción: arrow_up
....#.....
...###....
..#.#.#...
.#..#..#..
#...#...#.
....#.....
....#.....
....#.....
....#.....
....#.....

Entrada: triangle_yield -> Predicción: triangle_yield
.....#....
....###...
....#.##..
..#######.
.###